In [3]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

# Cargue de Queries y parametros

In [4]:
city = 'CMX'
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [5]:
margenes = pd.read_csv("Margenes_neto.csv")
retention = pd.read_csv("retention_year.csv")

In [6]:
retention_aux =  retention.copy()

for i,ret in enumerate(retention_aux.values):
    retention_aux.iloc[i,2:] = [(1-(j-ret[-1])/100) if j-ret[-1] >= 0 else 1 for j in ret[2:]]

In [7]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",157505,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [8]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND extract(year from bo.submit_date::date) = 2022
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    DATE(bo.submit_date) AS submit_date,
    bo.order_id,
    --bs.addl_product_id AS padre_sku_id,
    CASE
    WHEN bcat2.name = 'Abarrotes & Despensa' THEN 'Abarrotes'
    ELSE bcat2.name END AS cat,
    --bcat.name AS subcat,
    --bs2.name,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
    ELSE (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776 END AS gmv_usd,    
    -- CASE 
    --     WHEN s.site_identifier_value = 'CMX' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'GDL' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'PBC' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'MTY' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
    --     WHEN s.site_identifier_value = 'SPO' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'BHZ' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'CWB' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    --     WHEN s.site_identifier_value = 'VCP' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    -- ELSE (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/3776 END AS discount_usd,
    (boi.quantity*foi.step_unit) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd", "cant"]] = data_ventas_aux[["gmv_usd", "cant"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [9]:
data_ventas_aux

,city,customer_id,submit_date,order_id,cat,gmv_usd,cant
0,CMX,28734703,2022-01-10,6491147,Abarrotes,3.66412,5.0
1,CMX,51878683,2022-01-01,6422037,Frutas & Verduras,0.53994,1.0
2,CMX,39466505,2022-01-01,6556450,Abarrotes,37.86259,40.0
3,CMX,58008757,2022-01-02,6504833,Abarrotes,51.90839,3.0
4,CMX,74678694,2022-01-04,6539948,Aseo e Higiene,4.29312,2.0
...,...,...,...,...,...,...,...
4547848,CMX,280974020,2022-12-29,18104868,Aseo e Higiene,1.58066,1.0
4547849,CMX,-255913,2022-12-29,18105216,Aseo e Higiene,1.61068,1.0
4547850,CMX,180729932,2022-12-28,18109974,Lácteos & Huevos,2.07633,2.0
4547851,CMX,54157489,2022-12-31,18109535,Frutas & Verduras,3.77404,2.0


In [10]:
check_df = data_ventas_aux.groupby(by=['customer_id']).agg({'submit_date':np.max}).reset_index()
df = data_ventas_aux[data_ventas_aux.customer_id.isin(check_df[check_df.submit_date >= customer_date_desde].customer_id.unique())].groupby(by=['city', 'customer_id', 'submit_date','cat']).agg({'gmv_usd':'sum','cant':'sum'}).reset_index()

# Generacion de df con info by cat by client

In [11]:
df = df.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df['order_id']= df.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df['days_bet'] = (df.submit_date -df.groupby('customer_id').submit_date.transform('min')).dt.days

In [12]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df['res'] = df.groupby(by=['customer_id', 'cat'])['days_bet'].diff()
df_grouped = df.groupby(by=['city', 'customer_id', 'cat']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).reset_index()

In [13]:
#Finally we define everything in a df2 for further processing
df2 = df_grouped.copy()

df2.columns = ['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_cat_usd', 'avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [14]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [15]:
def cat_metrics_2(cat):
    df_cat = df2[df2.cat == cat].copy()
    
    df_cat['adjust_std'] = pd.qcut(df_cat.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
    df_cat[["adjust_std"]] = df_cat[["adjust_std"]].astype(float)
    
    df_cat['adjust_cohort'] = [1 if ((x[13]) - (x[5]+x[7]) <= 0) else  
                 1/(((x[13]) - x[5])/x[7]) if x[7] != 0 else
                 1/(((x[13]) - x[5])/np.round(1+(x[5]*0.7)))
                 for x in df_cat.values]
    
    df_cat["margen"] = [margenes[(margenes.category == x[2]) & (margenes.region_code == x[0])]["net_margin"].unique()[0]/100 for x in df_cat.values]
    #mirar aca
    df_cat['adjust_retention'] = [1 if diff_month(x[11],x[14])+1 >= 10 else 
                                  retention_aux[(retention_aux.category == x[2]) & (retention_aux.region_code == x[0])][str(diff_month(x[11],x[14])+1)].unique()[0] for x in df_cat.values]
    
    df_cat['median_diff_days_adjusted'] = 365/df_cat.median_diff_days
    
    df_cat["churned"] = [0 if ((x[13]) - (x[5]+x[7]*2) <= 0) else 1 for x in df_cat.values]
    
    df_cat[['avg_diff_days','stdev_diff_days']] = df_cat[['avg_diff_days','stdev_diff_days']].fillna(0)

    df_cat['churned_date'] = [(x[11] + datetime.timedelta(days=int(x[5]+x[7]*2))).strftime('%Y-%m-%d') for x in df_cat.values] #1 months
    
    df_cat["CLTV"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_margen"] = df_cat.median_diff_days_adjusted * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std * df_cat.adjust_cohort
    df_cat["CLTV_sin_churn"] = df_cat.median_diff_days_adjusted * df_cat.margen * df_cat.aov_cat_usd * df_cat.adjust_retention * df_cat.adjust_std
    
    df_cat['cash_margin'] = df_cat.aov_cat_usd * df_cat.margen

    return df_cat

In [16]:
df3 = pd.DataFrame()

for cat in df2.cat.unique():
   print(cat)
   if cat not in ('Mesa & Cocina','Insumos Internos Cat','Despensa'):
      df3 = df3.append(cat_metrics_2(cat))
   print(f'Done {cat}')

Abarrotes
Done Abarrotes
Aseo e Higiene
Done Aseo e Higiene
Bebidas
Done Bebidas
Carne, Pollo & Pescados
Done Carne, Pollo & Pescados
Desechables
Done Desechables
Frutas & Verduras
Done Frutas & Verduras
Lácteos & Huevos
Done Lácteos & Huevos
Congelados
Done Congelados


In [17]:
df3.head(5)

,city,customer_id,cat,min_diff_days,max_diff_days,avg_diff_days,median_diff_days,stdev_diff_days,num_orders,aov_cat_usd,...,adjust_cohort,margen,adjust_retention,median_diff_days_adjusted,churned,churned_date,CLTV,CLTV_sin_margen,CLTV_sin_churn,cash_margin
0,CMX,-269919,Abarrotes,1.0,13.0,3.205607,2.0,2.444626,108,16.188054,...,0.019589,0.041,1.000,182.500000,1,2022-12-28,2.372787,57.872843,121.127117,0.663710
7,CMX,-269912,Abarrotes,NaN,NaN,0.000000,NaN,0.000000,1,21.185740,...,NaN,0.041,0.580,NaN,1,2022-11-22,NaN,NaN,NaN,0.868615
10,CMX,-269908,Abarrotes,1.0,64.0,9.771429,7.0,12.586374,36,24.337170,...,0.105565,0.041,1.000,52.142857,1,2023-01-22,2.746244,66.981550,26.014696,0.997824
17,CMX,-269896,Abarrotes,8.0,64.0,28.333333,13.0,30.989245,4,16.593887,...,0.100723,0.041,0.749,28.076923,1,2022-08-24,0.288220,7.029752,2.861497,0.680349
23,CMX,-269885,Abarrotes,7.0,40.0,15.578947,13.0,9.708620,20,4.106072,...,0.066762,0.041,1.000,28.076923,1,2022-12-21,0.189340,4.618038,2.836032,0.168349


# Aca empieza el analisis global by customer 

In [18]:
#Here, i group the df3 (the df with the whole info by cat to customer lvl)
df_new3 = df3[['city','customer_id','aov_cat_usd','cash_margin','CLTV_sin_churn','churned']].groupby(by=['city','customer_id']).sum().reset_index()

In [19]:
#The processing data for customer level is made here

df_new = df.groupby(by=['city','customer_id','order_id','submit_date']).sum().reset_index()

df_new['days_bet'] = (df_new.submit_date -df_new.groupby('customer_id').submit_date.transform('min')).dt.days

#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df_new['res'] = df_new.groupby(by=['customer_id'])['days_bet'].diff()
df_grouped_2 = df_new.groupby(by=['city', 'customer_id']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                    'order_id': [lambda x: x.nunique()], 
                                                                    'gmv_usd': [np.mean],
                                                                    'cant': [np.mean],
                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                    }).droplevel(0, axis="columns").reset_index()

In [20]:
#Finally we define everything in a df_new2 at level customer for further processing
df_new2 = df_grouped_2.copy()

df_new2.columns = ['city', 'customer_id', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [21]:
#Finally we calculate the next parameters and also integred with df_new3 where we can have the info grouped by customer about the category performance
df_new2["churned_ALL"] = [0 if ((x[12]) - (x[4]+x[6]*2) <= 0) else 1 for x in df_new2.values]

df_new2['adjust_std'] = pd.qcut(df_new2.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
df_new2[["adjust_std"]] = df_new2[["adjust_std"]].astype(float)

df_new2['adjust_retention'] = [1 if diff_month(x[10],x[13])+1 >= 10 else 
                                retention_aux[(retention_aux.region_code == x[0])][str(diff_month(x[10],x[13])+1)].max() for x in df_new2.values]

df_new2['median_diff_days_adjusted'] = 365/df_new2.median_diff_days

#this merge is to include the info for the total cash_margin, how many cats is churned and the total cat CLTV
df_new2 = df_new2.merge(df_new3[['customer_id','cash_margin','CLTV_sin_churn','churned']], on='customer_id', how='left')

In [22]:
#We calculate now the CLTV taking into account the global behavior of the customer
df_new2["ALL_CLTV_sin_churn"] = df_new2.median_diff_days_adjusted * df_new2.adjust_retention * df_new2.adjust_std * df_new2.cash_margin / 2
df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']] = df_new2[['CLTV_sin_churn','ALL_CLTV_sin_churn']].fillna(0)

#df_new2["CLTV"] = df_new2.median_diff_days_adjusted * df_new2.margen * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort
#df_new2["CLTV_sin_margen"] = df_new2.median_diff_days_adjusted * df_new2.aov_cat_usd * df_new2.adjust_retention * df_new2.adjust_std * df_new2.adjust_cohort

#If the Total Cat CLTV is NA because it is so few orders for each category, we create a Adjust_CLTV to include the global or ALL_CLTV_sin_churn instead of the NULL value
df_new2['Adjust_CLTV'] = [x[21] if x[19] == 0 else x[19] for x in df_new2.values]

print(df_new2.Adjust_CLTV.sum(),df_new2.CLTV_sin_churn.sum())

4161679.6217834307 4140902.754415242


In [23]:
#Finally, we calculate the ALL_churned_date based on the general behavior of the customer
df_new2[['avg_diff_days','stdev_diff_days']] = df_new2[['avg_diff_days','stdev_diff_days']].fillna(0)
df_new2['ALL_churned_date'] = [(x[10] + datetime.timedelta(days=int(x[4]+x[6]*2))).strftime('%Y-%m-%d') for x in df_new2.values] #1 months

# Final processing of the file to export

In [24]:
#We will merge the df_new2 with the clear_buyers in order to filter by restaurants and take away kams (also to include the microsegment)
df_new2 = df_new2.merge(clear_buyers[clear_buyers.registered_city == city][['customer_id','business_type','microsegment_name','is_kam']], on='customer_id', how='left')
df_new2_end = df_new2[(df_new2.is_kam == 0)].copy() #df_new2[(df_new2.business_type == 'Restaurante') & (df_new2.is_kam == 0)].copy()

In [25]:
#This cell is to calculate the cumulative sum to create the tiers further
df_new2_end = df_new2_end.sort_values(['Adjust_CLTV'], ascending=[False])
df_new2_end['cumsum'] = df_new2_end.groupby('city')['Adjust_CLTV'].cumsum()
df_new2_end['run_pct'] = df_new2_end.groupby('city')['Adjust_CLTV'].apply(lambda x: (x/x.sum()).cumsum())

In [26]:
#This conditional create the tiers
df_new2_end['tier'] = ['A' if x <= 0.1 else
                       'B' if x <= 0.2 else
                       'C' if x <= 0.3 else
                       'D' if x <= 0.4 else
                       'E' if x <= 0.5 else
                       'F' if x <= 0.6 else
                       'G' if x <= 0.7 else
                       'H' if x <= 0.8 else
                       'I' if x <= 0.9 else
                       'J' for x in df_new2_end.run_pct.values]

In [27]:
df_new2_end.tier.value_counts()

J    11697
I     3250
H     2168
G     1579
F     1184
E      904
D      673
C      492
B      335
A      173
Name: tier, dtype: int64

In [28]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

# df3 = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders',
#        'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
#        'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
#        'median_diff_days_adjusted', 'churned', 'churned_date','CLTV_sin_churn']].copy()

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [29]:
df_new2_end.columns

Index(['city', 'customer_id', 'min_diff_days', 'max_diff_days',
       'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
       'ALL_num_orders', 'aov_ALL_usd', 'ALL_avg_cant', 'last_order_date',
       'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'churned_ALL', 'adjust_std', 'adjust_retention',
       'median_diff_days_adjusted', 'cash_margin', 'TOTAL_CAT_CLTV_sin_churn',
       'churned', 'ALL_CLTV_sin_churn', 'Adjust_CLTV', 'ALL_churned_date',
       'business_type', 'microsegment_name', 'is_kam', 'cumsum', 'run_pct',
       'tier'],
      dtype='object')

In [30]:
df_new2_end.rename(columns={'num_orders': 'ALL_num_orders', 'CLTV_sin_churn': 'TOTAL_CAT_CLTV_sin_churn'}, inplace=True)

# df3 = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders',
#        'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
#        'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
#        'median_diff_days_adjusted', 'churned', 'churned_date','CLTV_sin_churn']].copy()

df3.rename(columns={'num_orders': 'num_orders_cat', 'churned': 'churned_cat','CLTV_sin_churn':'CAT_CLTV_sin_churn'}, inplace=True)

In [31]:
df_final = df3[['city', 'customer_id', 'cat', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days', 'num_orders_cat',
       'aov_cat_usd','last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order',
       'first_order_date', 'adjust_std', 'adjust_cohort','adjust_retention', 
       'median_diff_days_adjusted', 'churned_cat', 'churned_date','CAT_CLTV_sin_churn']].merge(df_new2_end, on='customer_id', how='inner')

In [32]:
df_final.head(10)

,city_x,customer_id,cat,min_diff_days_x,max_diff_days_x,avg_diff_days_x,median_diff_days_x,stdev_diff_days_x,num_orders_cat,aov_cat_usd,...,churned,ALL_CLTV_sin_churn,Adjust_CLTV,ALL_churned_date,business_type,microsegment_name,is_kam,cumsum,run_pct,tier
0,CMX,-269919,Abarrotes,1.0,13.0,3.205607,2.0,2.444626,108,16.188054,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
1,CMX,-269919,Aseo e Higiene,1.0,36.0,12.481481,11.0,8.802842,28,15.059532,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
2,CMX,-269919,Bebidas,NaN,NaN,0.000000,NaN,0.000000,1,13.884980,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
3,CMX,-269919,"Carne, Pollo & Pescados",1.0,25.0,4.118644,3.0,4.077206,60,14.838688,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
4,CMX,-269919,Desechables,1.0,33.0,10.413793,10.0,7.514354,30,7.535361,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
5,CMX,-269919,Frutas & Verduras,1.0,13.0,2.731707,2.0,2.048883,124,20.903337,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
6,CMX,-269919,Lácteos & Huevos,1.0,23.0,5.276923,5.0,4.536338,66,20.924845,...,7,954.964187,1104.693302,2022-12-26,Restaurante,Fonda,False,4.605577e+05,0.134216,B
7,CMX,-269912,Abarrotes,NaN,NaN,0.000000,NaN,0.000000,1,21.185740,...,3,0.000000,0.000000,2023-08-16,Comercio,None,False,3.431459e+06,1.000000,J
8,CMX,-269912,Aseo e Higiene,NaN,NaN,0.000000,NaN,0.000000,1,0.489560,...,3,0.000000,0.000000,2023-08-16,Comercio,None,False,3.431459e+06,1.000000,J
9,CMX,-269912,Frutas & Verduras,267.0,267.0,267.000000,267.0,0.000000,2,1.652665,...,3,0.000000,0.000000,2023-08-16,Comercio,None,False,3.431459e+06,1.000000,J


# Info for ENAE

In [33]:
data_ventas_aux

,city,customer_id,submit_date,order_id,cat,gmv_usd,cant
0,CMX,28734703,2022-01-10,6491147,Abarrotes,3.66412,5.0
1,CMX,51878683,2022-01-01,6422037,Frutas & Verduras,0.53994,1.0
2,CMX,39466505,2022-01-01,6556450,Abarrotes,37.86259,40.0
3,CMX,58008757,2022-01-02,6504833,Abarrotes,51.90839,3.0
4,CMX,74678694,2022-01-04,6539948,Aseo e Higiene,4.29312,2.0
...,...,...,...,...,...,...,...
4547848,CMX,280974020,2022-12-29,18104868,Aseo e Higiene,1.58066,1.0
4547849,CMX,-255913,2022-12-29,18105216,Aseo e Higiene,1.61068,1.0
4547850,CMX,180729932,2022-12-28,18109974,Lácteos & Huevos,2.07633,2.0
4547851,CMX,54157489,2022-12-31,18109535,Frutas & Verduras,3.77404,2.0


In [34]:
df_enae = data_ventas_aux.merge(df_final[['customer_id','Adjust_CLTV']].drop_duplicates(), on='customer_id', how='inner')

In [35]:
df_enae

,city,customer_id,submit_date,order_id,cat,gmv_usd,cant,Adjust_CLTV
0,CMX,28734703,2022-01-10,6491147,Abarrotes,3.66412,5.00,480.380128
1,CMX,28734703,2022-01-17,6849171,Lácteos & Huevos,17.00152,1.00,480.380128
2,CMX,28734703,2022-02-07,7431850,Abarrotes,43.01374,1.00,480.380128
3,CMX,28734703,2022-04-25,9906793,"Carne, Pollo & Pescados",5.80966,2.00,480.380128
4,CMX,28734703,2022-05-30,11087228,Desechables,22.68702,3.00,480.380128
...,...,...,...,...,...,...,...,...
3259425,CMX,49314249,2022-12-20,17871079,"Carne, Pollo & Pescados",36.92071,9.08,0.000000
3259426,CMX,83109056,2022-12-17,17927825,"Carne, Pollo & Pescados",32.10000,5.35,0.000000
3259427,CMX,194498280,2022-12-22,17849928,Abarrotes,52.20610,1.00,0.000000
3259428,CMX,272756888,2022-12-13,17662183,"Carne, Pollo & Pescados",51.40763,6.00,0.000000


In [36]:
df_enae.to_csv(f"{city}_ENAE_info.csv",index=False)

In [362]:
df_SAC = df_final[['city','customer_id','tier','Adjust_CLTV']].drop_duplicates()

In [363]:
letter_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10}
df_SAC['tier'] = df_SAC['tier'].map(letter_to_number)

In [364]:
df_SAC.to_csv(f"{city}_SAC_tiers.csv",index=False)